In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "coat"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-3,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

# L = 5, 20%, 19.66%
# L = 6.7, 15%, 15.05%
# L = 3.2, 30%, 30.12%
# L = 9.7, 10%, 10.02%
# L = 3.92, 25%, 24.99%

L = 6.7
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test = feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
mask proportion (within the original y_train = 1 )
15.046935394809497
prediction for base model
0.6076920935476612


In [2]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.025, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_IPS_DF] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])
print(mf_ips_df_mean)
print(mf_ips_df_sd)

[MF-IPS-DF] epoch:32, xent:566.5209560394287
0.7151511337739598
[MF-IPS-DF] epoch:32, xent:568.6516065597534
0.7142189204885091
[MF-IPS-DF] epoch:34, xent:567.8937559127808
0.7185610972819274
[MF-IPS-DF] epoch:30, xent:568.7042140960693
0.7202264957364098
[MF-IPS-DF] epoch:24, xent:566.4302568435669
0.7127508102251927
[MF-IPS-DF] epoch:40, xent:566.6337699890137
0.7133943892437048
[MF-IPS-DF] epoch:33, xent:567.0827007293701
0.7138997408671324
[MF-IPS-DF] epoch:34, xent:565.9329051971436
0.721771259373364
[MF-IPS-DF] epoch:35, xent:566.631178855896
0.7166063879229081
[MF-IPS-DF] epoch:30, xent:567.0550403594971
0.7175137589422491
[MF_IPS_DF] test auc: 0.7164093993855357  sd:  0.002904537179289281
[0.2283225  0.7164094  0.61564213 0.68108243 0.41847722 0.69477173]
[0.0003797  0.00290454 0.00764206 0.00689366 0.00405468 0.00460747]


In [3]:
# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])
print(mf_dr_df_mean)
print(mf_dr_df_sd)

[MF-DR-JL-DF] epoch:22, xent:52012526.125
0.7206010629783345
[MF-DR-JL-DF] epoch:23, xent:51876946.0
0.7179469036676851
[MF-DR-JL-DF] epoch:23, xent:51995718.3125
0.7178254955500445
[MF-DR-JL-DF] epoch:23, xent:52047707.75
0.7207402569985594
[MF-DR-JL-DF] epoch:21, xent:51978985.6875
0.7281092270942707
[MF-DR-JL-DF] epoch:27, xent:51978647.125
0.739208306944467
[MF-DR-JL-DF] epoch:22, xent:51944015.75
0.7156860641266852
[MF-DR-JL-DF] epoch:17, xent:52066851.375
0.7190313604127566
[MF-DR-JL-DF] epoch:21, xent:51980233.0625
0.7259209811013185
[MF-DR-JL-DF] epoch:20, xent:51996423.5625
0.7396698704490321
[MF_DR_JL_DF] test auc: 0.7244739529323154  sd:  0.008282023476584622
[0.33801535 0.72447395 0.62154348 0.69253478 0.43809142 0.72420532]
[0.00062958 0.00828202 0.01695671 0.01366585 0.01461087 0.00823181]


In [4]:
# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])
print(mlp_weibull_mean)
print(mlp_weibull_sd)

[MLP_weibull] epoch:18, xent:50.197541892528534
0.6325795010513016
[MLP_weibull] epoch:21, xent:50.1369903087616
0.6326038600048408
[MLP_weibull] epoch:18, xent:50.13792127370834
0.6325115473039278
[MLP_weibull] epoch:20, xent:50.133437633514404
0.6324808086244615
[MLP_weibull] epoch:19, xent:50.096117436885834
0.6325718647126919
[MLP_weibull] epoch:18, xent:50.12433344125748
0.6325934204533241
[MLP_weibull] epoch:17, xent:50.11892652511597
0.6325042976153744
[MLP_weibull] epoch:19, xent:50.142668306827545
0.6325916805280711
[MLP_weibull] epoch:17, xent:50.09222108125687
0.6325621984612875
[MLP_weibull] epoch:15, xent:50.161166071891785
0.6326173927568072
[MLP_weibull] test auc: 0.6325616571512087  sd:  4.42248104880139e-05
[0.23754495 0.63256166 0.52612796 0.60560667 0.36020046 0.6452946 ]
[2.54511778e-03 4.42248105e-05 1.95921205e-04 1.22088465e-04
 2.81550545e-04 1.58019852e-04]


In [5]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])
print(mf_dr_mean)
print(mf_dr_sd)

[MF-DR] epoch:15, xent:57148.44891357422
0.7005190390354163
[MF-DR] epoch:15, xent:57157.68493652344
0.6925076498713615
[MF-DR] epoch:13, xent:57146.94396972656
0.7095974856920148
[MF-DR] epoch:16, xent:57123.3349609375
0.6999086152592218
[MF-DR] epoch:14, xent:57168.62756347656
0.7003116979427896
[MF-DR] epoch:18, xent:57123.33508300781
0.6876536450660746
[MF-DR] epoch:16, xent:57186.236572265625
0.6697338648998306
[MF-DR] epoch:14, xent:57117.155029296875
0.6856196724455385
[MF-DR] epoch:14, xent:57144.57263183594
0.6868058181553854
[MF-DR] epoch:12, xent:57112.076904296875
0.6927887444622045
[MF_DR] test auc: 0.6925446232829838  sd:  0.010460159683488828
[0.23547919 0.69254462 0.58448861 0.6612974  0.3969792  0.69350402]
[0.00029467 0.01046016 0.0138009  0.00735882 0.01274147 0.01060449]
